<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Common Functions for `GiRaFFEfood` Initial Data for `GiRaFFE`

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Common_Functions.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Common_Functions.py)

**Notebook Status:** <font color='red'><b> In Progress </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module through the main initial data modules that depend on it.

## Introduction: 
We will need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; while each represents different physical systems, they all have some steps in common with each other. To avoid code duplication, we will first write several functions that we will use for all of them.

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#initializenrpy): Compute vector potential from input 
1. [Step 3](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 4](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters


In [ ]:
# Step 0: Import the NRPy+ core modules and set the reference metric to Cartesian
import NRPy_param_funcs as par
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = __name__


In [ ]:
# Generic function for all 1D tests: Compute Ax,Ay,Az
def Axyz_func(Ax_func,Ay_func,Az_func, stagger_enable, **params):
    x = rfm.xxCart[0]
    y = rfm.xxCart[1]
    z = rfm.xxCart[2]
    # First Ax
    if stagger_enable:
        y += sp.Rational(1,2)*gri.dxx[1]
        z += sp.Rational(1,2)*gri.dxx[2]
    Ax = Ax_func(x,y,z, **params)
    # Then Ay
    if stagger_enable:
        x += sp.Rational(1,2)*gri.dxx[0]
        z += sp.Rational(1,2)*gri.dxx[2]
    Ay = Ay_func(x,y,z, **params)
    # Finally Az
    if stagger_enable:
        x += sp.Rational(1,2)*gri.dxx[0]
        y += sp.Rational(1,2)*gri.dxx[1]
    Az = Az_func(x,y,z, **params)

    return Ax,Ay,Az

In [ ]:
# Generic function for all 1D tests: Valencia 3-velocity from EU and BU
def compute_ValenciavU_from_EU_and_BU(EU, BU, gammaDD=None):
    # Now, we calculate v^i = ([ijk] E^j B^k) / B^2,
    # where [ijk] is the Levi-Civita symbol and B^2 = \gamma_{ij} B^i B^j$ is a trivial dot product in flat space.
    LeviCivitaSymbolDDD = ixp.LeviCivitaSymbol_dim3_rank3()

    B2 = sp.sympify(0)
    # In flat spacetime, use the Minkowski metric; otherwise, use the input metric.
    if gammaDD==None:
        gammaDD = ixp.zerorank2()
        for i in range(3):
            gammaDD[i][i] = sp.sympify(1)
    for i in range(3):
        for j in range(3):
            B2 += gammaDD[i][j] * BU[i] * BU[j]

    ValenciavU = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                ValenciavU[i] += LeviCivitaSymbolDDD[i][j][k] * EU[j] * BU[k] / B2

    return ValenciavU

In [ ]:
# def GiRaFFEfood_NRPy_1D_tests(ID_type = "DegenAlfvenWave", stagger_enable = False):
#     global AD, ValenciaVU
#     AD = ixp.zerorank1()
#     if ID_type == "DegenAlfvenWave":
#         mu_DAW = par.Cparameters("REAL",thismodule,["mu_DAW"], -0.5) # The wave speed
#         AD = Axyz_func(Ax_DAW, Ay_DAW, Az_DAW, stagger_enable,
#                        gammamu=sp.sympify(1)/sp.sqrt(sp.sympify(1)-mu_AW**2))
#         ValenciaVU = ValenciavU_DAW(mu_DAW=mu_DAW, gammamu=gammamu)
#     elif ID_type == "FastWave":
#         AD = Axyz_func(Ax_FW, Ay_FW, Az_FW, stagger_enable)
#         ValenciaVU = ValenciavU_FW()